In [61]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()


#carregar tabela ( atenção ao caminho)
arquivo_json_grande = spark.read.json('s3a://staack-yuiti-b/casos_obitos_doencas_pre_multiline.json')
arquivo_csv = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://staack-yuiti-b/evolucao_caso.csv')
print(arquivo_json_grande.count())
print(arquivo_csv.count())

print( type(arquivo_json_grande) )
print( type(arquivo_csv) )


1000001


1000000
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [37]:
#exibir a tabela
arquivo_json.printSchema()
arquivo_csv.printSchema()


root
 |-- asma: string (nullable = true)
 |-- cardiopatia: string (nullable = true)
 |-- codigo_ibge: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- data_inicio_sintomas: string (nullable = true)
 |-- diabetes: string (nullable = true)
 |-- diagnostico_covid19: string (nullable = true)
 |-- doenca_hematologica: string (nullable = true)
 |-- doenca_hepatica: string (nullable = true)
 |-- doenca_neurologica: string (nullable = true)
 |-- doenca_renal: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- imunodepressao: string (nullable = true)
 |-- obesidade: string (nullable = true)
 |-- obito: string (nullable = true)
 |-- outros_fatores_de_risco: string (nullable = true)
 |-- pneumopatia: string (nullable = true)
 |-- puerpera: string (nullable = true)
 |-- sindrome_de_down: string (nullable = true)

root
 |-- source_id: string (nullable = true)
 |-- dataNotificacao: string (nullable = true)
 |-- dataInicioSinto

In [63]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

json_pre_tratado = arquivo_json_grande.select(
    col('estado').alias('ESTADO'),
    col('cs_sexo').alias('SEXO'),
    col('idade').cast(IntegerType()).alias('IDADE'),
    
    col('asma').alias('ASMA'),
    col('cardiopatia').alias('CARDIOPATIA'),
    col('diabetes').alias('DIABETE'),
    col('obesidade').alias('OBESIDADE'),
    col('sindrome_de_down').alias('SIN_DOWN')
)

csv_pre_tratado = arquivo_csv.select(
    col('estadoIBGE').alias('ESTADO'),
    col('sexo').alias('SEXO'),
    col('idade').cast(IntegerType()).alias('IDADE'),
    
    col('racaCor').alias('ETNIA'),
    col('evolucaoCaso').alias('EVOLUCAO_CASO'),
    col('codigoLaboratorioPrimeiraDose').alias('PRIMEIRA_DOSE'),
    col('codigoLaboratorioSegundaDose').alias('SEGUNDA_DOSE')
)

json_pre_tratado.printSchema()

csv_pre_tratado.printSchema()

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: integer (nullable = true)
 |-- ETNIA: string (nullable = true)
 |-- EVOLUCAO_CASO: string (nullable = true)
 |-- PRIMEIRA_DOSE: string (nullable = true)
 |-- SEGUNDA_DOSE: string (nullable = true)



In [72]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

arquivo_json_grande.write.mode('overwrite').option("header", "true").csv('s3a://stack-yuit-test-sptech-data-transform/preTratedJson4.csv')
#json_pre_tratado.write.csv('s3a://stack-yuit-test-sptech-data-transform/preTratedJson.csv', mode = "overwrite")
#json_pre_tratado.coalesce(1).write.format('csv').option('header','true').save('s3a://stack-yuit-test-sptech-data-transform/preTratedJson2.csv',mode='overwrite')
#json_pre_tratado.write.format('csv').option('header','true').save('preTratedJson.csv',mode='overwrite')

#arquivo_json_grande.write.options("header","true").csv("s3a://stack-yuit-test-sptech-data-transform/preTratedJson4")

print("salvo")

22/05/28 00:36:56 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/28 00:36:56 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/28 00:36:56 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/28 00:36:56 ERROR Executor: Exception in task 0.0 in stage 240.0 (TID 262)
java.lang.NoSuchMethodError: 'void org.apache.hadoop.util.SemaphoredDelegatingExecutor.<init>(com.google.common.util.concurrent.ListeningExecutorService, int, boolean)'
	at org.apache.hadoop.fs.s3a.S3AFileSystem.create(S3AFileSystem.java:772)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.

Py4JJavaError: An error occurred while calling o1827.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at org.a

In [50]:

#fazendo calculos
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType
json_pre_tratado.select(sum(col('idade').cast(IntegerType())).alias('soma')).show()
csv_pre_tratado.select(sum(col('idade').cast(IntegerType())).alias('soma')).show()


SyntaxError: invalid syntax (1121803564.py, line 4)

In [ ]:


# select via id

from pyspark.sql.functions import col
a.select(col('id_cliente')).first()

In [55]:
json_pre_tratado.groupby('estado').avg().show()
csv_pre_tratado.groupby('estado').avg().show()

+------+------------------+
|estado|        avg(IDADE)|
+------+------------------+
|    SC|39.297366896285325|
|    RO| 39.24902925187678|
|    PI| 39.35966872617326|
|    AM|  39.7020710401481|
|    RR| 40.86971830985915|
|    GO| 39.33841586830387|
|  null|              null|
|    TO|38.947944395149364|
|    MT|39.551067961165046|
|    SP|39.577979750133224|
|    PB|39.438258232235704|
|    ES|39.639233894370285|
|    RS| 39.47241300366301|
|    MS| 39.46501789781972|
|    AL| 39.50510930021989|
|    MG| 39.46096800997092|
|    PA|39.362112720735965|
|    BA| 39.41946211683054|
|    SE| 39.59706744868035|
|    PE|39.607457469121414|
+------+------------------+
only showing top 20 rows



+------+------------------+
|estado|        avg(IDADE)|
+------+------------------+
|    SC| 35.85145317545748|
|    RO|35.515151515151516|
|    PI| 37.00729927007299|
|    AM| 35.82213438735178|
|    RR|23.545454545454547|
|    GO| 35.25589836660617|
|  null|38.992098935073855|
|    TO| 37.05714285714286|
|    MT| 35.66554054054054|
|    SP|36.169973792978105|
|    PB|37.172284644194754|
|    ES|  36.7906976744186|
|    RS| 37.74193548387097|
|    MS| 35.38107416879795|
|    AL| 32.14035087719298|
|    MG| 36.77768595041322|
|    PA| 38.37238493723849|
|    BA|  36.4173140954495|
|    SE| 37.36974789915966|
|    PE|  37.7910284463895|
+------+------------------+
only showing top 20 rows

